# Elo Model Training & Hyperparameter Tuning

This notebook:
1. Loads WHL 2025 data and aggregates shifts → games (sum for goals)
2. Runs grid search over Elo hyperparameters (K-factor, home advantage, MOV, rest, B2B)
3. Fine-tunes top parameters with a narrow sweep
4. Evaluates with combined RMSE (home + away), win accuracy — comparable to baseline
5. Generates Round 1 predictions from `WHSDSC_Rnd1_matchups.xlsx`
6. Saves all outputs to `output/predictions/elo/` and `output/models/elo/`

**Baseline benchmark: Ensemble RMSE 1.8071, Win Acc 55.9%**

In [ ]:
import pandas as pd
import numpy as np
from itertools import product
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns
import json, os, sys, pathlib, pickle

# ---------- resolve python/ directory & set CWD ----------
_cwd = pathlib.Path(os.path.abspath('')).resolve()
if (_cwd / 'python').is_dir():
    _python_dir = _cwd / 'python'                          # workspace root
elif _cwd.name == 'elo' and (_cwd.parent.parent / 'data').is_dir():
    _python_dir = _cwd.parent.parent                       # training/elo/
elif _cwd.name == 'training' and (_cwd.parent / 'data').is_dir():
    _python_dir = _cwd.parent                              # training/
elif (_cwd / 'data').is_dir():
    _python_dir = _cwd                                     # already in python/
else:
    raise RuntimeError(f'Cannot locate python/ directory from {_cwd}')

os.chdir(_python_dir)
sys.path.insert(0, str(_python_dir))

from utils.elo_model import EloModel

sns.set_style('darkgrid')
plt.rcParams['figure.figsize'] = (12, 6)
print(f'CWD: {os.getcwd()}')

In [ ]:
# ── Load & aggregate raw shift-level data to game level ──
raw = pd.read_csv('data/whl_2025.csv')
print(f'Raw: {len(raw)} shift rows')

games_df = raw.groupby('game_id').agg(
    home_team  = ('home_team', 'first'),
    away_team  = ('away_team', 'first'),
    home_goals = ('home_goals', 'sum'),
    away_goals = ('away_goals', 'sum'),
    went_ot    = ('went_ot', 'max'),
).reset_index()

# Sort by game_id (chronological)
games_df['game_num'] = games_df['game_id'].str.extract(r'(\d+)').astype(int)
games_df = games_df.sort_values('game_num').reset_index(drop=True)

print(f'Games: {len(games_df)}')
print(f'Teams: {games_df["home_team"].nunique()}')
print(f'Avg home goals: {games_df["home_goals"].mean():.2f}, Avg away goals: {games_df["away_goals"].mean():.2f}')
print(f'Home win rate: {(games_df["home_goals"] > games_df["away_goals"]).mean():.1%}')
games_df.head()

## Train/Test Split & Grid Search Setup

In [ ]:
# 80/20 chronological split (same as baseline)
split_idx = int(len(games_df) * 0.8)
train_df = games_df.iloc[:split_idx].copy()
test_df  = games_df.iloc[split_idx:].copy()

print(f'Train: {len(train_df)} games  |  Test: {len(test_df)} games')

# ── Build parameter grid ──
param_grid = dict(
    k_factor              = [20, 32, 40],
    home_advantage        = [50, 100, 150],
    mov_multiplier        = [0.0, 1.0, 1.5],
    mov_method            = ['logarithmic'],      # keep constant for grid
    initial_rating        = [1500],
    season_carryover      = [0.75],               # single-season data
    ot_win_multiplier     = [0.75],
    rest_advantage_per_day= [0, 10],
    b2b_penalty           = [0, 50],
)

# Generate all combos
keys = list(param_grid.keys())
combos = list(product(*[param_grid[k] for k in keys]))
print(f'Grid: {len(combos)} configurations')

In [ ]:
# ── Grid Search ──
results = []

for i, vals in enumerate(combos):
    params = dict(zip(keys, vals))
    label = f"Elo(k={params['k_factor']},ha={params['home_advantage']},mov={params['mov_multiplier']},rest={params['rest_advantage_per_day']},b2b={params['b2b_penalty']})"
    
    try:
        model = EloModel(params)
        model.fit(train_df)
        metrics = model.evaluate(test_df)
        results.append({'config': label, **params, **metrics})
    except Exception as e:
        print(f'  FAILED {label}: {e}')

    if (i + 1) % 20 == 0:
        print(f'  {i+1}/{len(combos)} done')

results_df = pd.DataFrame(results).sort_values('combined_rmse')
print(f'\n✅ {len(results_df)} configs evaluated')
print(f'Best combined RMSE: {results_df["combined_rmse"].min():.4f}')
print(f'Win accuracy range: {results_df["win_accuracy"].min():.1%} – {results_df["win_accuracy"].max():.1%}')
results_df.head(10)[['config','home_rmse','away_rmse','combined_rmse','win_accuracy']]

## Save Grid Search Results

In [ ]:
# Save full comparison (like baseline_comparison.csv)
os.makedirs('output/predictions/elo', exist_ok=True)
os.makedirs('output/models/elo', exist_ok=True)

results_df.to_csv('output/predictions/elo/elo_comparison.csv', index=False)
print(f'✅ Saved {len(results_df)} configs → output/predictions/elo/elo_comparison.csv')
print(f'\nTop 5:')
print(results_df.head()[['config','combined_rmse','win_accuracy']].to_string(index=False))

## Hyperparameter Sensitivity Analysis

In [ ]:
# ── Which hyperparameters matter most? ──
param_cols = ['k_factor', 'home_advantage', 'mov_multiplier', 'rest_advantage_per_day', 'b2b_penalty']

fig, axes = plt.subplots(1, len(param_cols), figsize=(18, 4), sharey=True)
for ax, col in zip(axes, param_cols):
    grouped = results_df.groupby(col)['combined_rmse'].mean()
    ax.bar(range(len(grouped)), grouped.values, tick_label=[str(v) for v in grouped.index])
    ax.set_title(col, fontsize=10)
    ax.set_ylabel('Mean Combined RMSE' if col == param_cols[0] else '')
plt.suptitle('Hyperparameter Sensitivity (lower = better)', fontsize=13)
plt.tight_layout()
plt.show()

# Correlation with RMSE
corr = results_df[param_cols + ['combined_rmse']].corr()['combined_rmse'].drop('combined_rmse')
print('Correlation with combined_rmse:')
for p, c in corr.abs().sort_values(ascending=False).items():
    direction = '↑' if corr[p] > 0 else '↓'
    print(f'  {p:30s}  r={corr[p]:+.3f}  ({direction} higher param → {"worse" if corr[p]>0 else "better"} RMSE)')

## Fine-Grained Hyperparameter Sweep

In [ ]:
# Narrow sweep around best grid values
best_row = results_df.iloc[0]
print(f'Grid search best: combined_rmse={best_row["combined_rmse"]:.4f}')
print(f'  k_factor={best_row["k_factor"]}, home_advantage={best_row["home_advantage"]}, '
      f'mov={best_row["mov_multiplier"]}, rest={best_row["rest_advantage_per_day"]}, b2b={best_row["b2b_penalty"]}')

# Fine sweep around best values
best_k  = int(best_row['k_factor'])
best_ha = int(best_row['home_advantage'])
best_mov = float(best_row['mov_multiplier'])
best_rest = int(best_row['rest_advantage_per_day'])
best_b2b = int(best_row['b2b_penalty'])

fine_grid = dict(
    k_factor              = list(range(max(10, best_k - 10), best_k + 12, 2)),
    home_advantage        = list(range(max(0, best_ha - 50), best_ha + 60, 10)),
    mov_multiplier        = [max(0, best_mov - 0.5), best_mov - 0.25, best_mov, best_mov + 0.25, best_mov + 0.5],
    mov_method            = ['logarithmic'],
    initial_rating        = [1500],
    season_carryover      = [0.75],
    ot_win_multiplier     = [0.75],
    rest_advantage_per_day= [max(0, best_rest - 5), best_rest, best_rest + 5],
    b2b_penalty           = [max(0, best_b2b - 25), best_b2b, best_b2b + 25],
)

fine_keys = list(fine_grid.keys())
fine_combos = list(product(*[fine_grid[k] for k in fine_keys]))
print(f'\nFine sweep: {len(fine_combos)} configurations')

## Run Fine Sweep

In [ ]:
fine_results = []

for i, vals in enumerate(fine_combos):
    params = dict(zip(fine_keys, vals))
    try:
        model = EloModel(params)
        model.fit(train_df)
        metrics = model.evaluate(test_df)
        fine_results.append({**params, **metrics})
    except Exception as e:
        pass
    if (i + 1) % 100 == 0:
        print(f'  {i+1}/{len(fine_combos)} done')

fine_df = pd.DataFrame(fine_results).sort_values('combined_rmse')
print(f'\n✅ Fine sweep done — {len(fine_df)} configs')

best_fine = fine_df.iloc[0]
print(f'\nBest from fine sweep:')
print(f'  combined_rmse = {best_fine["combined_rmse"]:.4f}')
print(f'  home_rmse     = {best_fine["home_rmse"]:.4f}')
print(f'  away_rmse     = {best_fine["away_rmse"]:.4f}')
print(f'  win_accuracy  = {best_fine["win_accuracy"]:.1%}')
print(f'  k_factor={best_fine["k_factor"]}, home_advantage={best_fine["home_advantage"]}, '
      f'mov={best_fine["mov_multiplier"]}, rest={best_fine["rest_advantage_per_day"]}, b2b={best_fine["b2b_penalty"]}')

# Compare to baseline
print(f'\n📊 vs Baseline ensemble: 1.8071 RMSE, 55.9% win acc')
improvement = 1.8071 - best_fine['combined_rmse']
print(f'   Elo improvement: {improvement:+.4f} RMSE ({"BETTER" if improvement > 0 else "not yet beating baseline"})')

In [ ]:
# ── Visualize fine sweep results ──
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# RMSE distribution
axes[0].hist(fine_df['combined_rmse'].dropna(), bins=40, edgecolor='black', alpha=0.7)
axes[0].axvline(best_fine['combined_rmse'], color='red', ls='--', lw=2,
                label=f'Best: {best_fine["combined_rmse"]:.4f}')
axes[0].axvline(1.8071, color='green', ls='--', lw=2, label='Baseline: 1.8071')
axes[0].set_xlabel('Combined RMSE')
axes[0].set_title('Fine Sweep RMSE Distribution')
axes[0].legend()

# K-factor vs RMSE
k_rmse = fine_df.groupby('k_factor')['combined_rmse'].mean()
axes[1].plot(k_rmse.index, k_rmse.values, 'o-', color='steelblue')
axes[1].set_xlabel('K-factor')
axes[1].set_ylabel('Mean Combined RMSE')
axes[1].set_title('K-factor Sensitivity')

plt.tight_layout()
plt.show()

## Train Final Model & Team Rankings

In [ ]:
# ── Train final model on ALL data with best params ──
best_params = {k: best_fine[k] for k in fine_keys}
# Clean up numpy types
best_params = {k: (int(v) if isinstance(v, (np.integer,)) else
                   float(v) if isinstance(v, (np.floating,)) else v)
               for k, v in best_params.items()}

final_model = EloModel(best_params)
final_model.fit(games_df)

# Also evaluate on the test split for recording
eval_model = EloModel(best_params)
eval_model.fit(train_df)
final_metrics = eval_model.evaluate(test_df)

print('Final Elo model trained on all', len(games_df), 'games')
print(f'\nTest-set metrics:')
print(f'  Home RMSE:     {final_metrics["home_rmse"]:.4f}')
print(f'  Away RMSE:     {final_metrics["away_rmse"]:.4f}')
print(f'  Combined RMSE: {final_metrics["combined_rmse"]:.4f}')
print(f'  Win Accuracy:  {final_metrics["win_accuracy"]:.1%}')

print(f'\n🏆 Team Rankings (top 10):')
for rank, (team, rating) in enumerate(final_model.get_rankings(10), 1):
    print(f'  {rank:2d}. {team:20s}  {rating:.1f}')

In [ ]:
# ── Rating distribution visualization ──
rankings = final_model.get_rankings()
teams_r, ratings_r = zip(*rankings)

fig, ax = plt.subplots(figsize=(12, 6))
colors = ['#2ecc71' if r > 1500 else '#e74c3c' if r < 1500 else '#95a5a6' for r in ratings_r]
ax.barh(range(len(teams_r)), ratings_r, color=colors, edgecolor='black', linewidth=0.3)
ax.set_yticks(range(len(teams_r)))
ax.set_yticklabels(teams_r, fontsize=9)
ax.axvline(1500, color='gray', ls='--', lw=1, label='Initial (1500)')
ax.set_xlabel('Elo Rating')
ax.set_title('Final Elo Ratings — All 32 Teams')
ax.legend()
ax.invert_yaxis()
plt.tight_layout()
plt.show()

## Round 1 Predictions

In [ ]:
# ── Load Round 1 matchups and predict ──
matchups = pd.read_excel('data/WHSDSC_Rnd1_matchups.xlsx')
print(f'{len(matchups)} matchups loaded')

home_col = [c for c in matchups.columns if 'home' in c.lower()][0]
away_col = [c for c in matchups.columns if 'away' in c.lower()][0]

preds = []
for i, row in matchups.iterrows():
    game = {'home_team': row[home_col], 'away_team': row[away_col]}
    h_goals, a_goals = final_model.predict_goals(game)
    winner, conf = final_model.predict_winner(game)
    preds.append({
        'game': i + 1,
        'home_team': row[home_col],
        'away_team': row[away_col],
        'pred_home_goals': round(h_goals, 2),
        'pred_away_goals': round(a_goals, 2),
        'predicted_winner': winner,
        'confidence': round(conf, 4),
    })

pred_df = pd.DataFrame(preds)
print(f'\nRound 1 Elo Predictions:')
print(pred_df.to_string(index=False))
print(f'\nAvg confidence: {pred_df["confidence"].mean():.1%}')

## Save All Outputs

In [ ]:
# ── 1. Predictions CSV ──
pred_df.to_csv('output/predictions/elo/round1_elo_predictions.csv', index=False)
print('✅ output/predictions/elo/round1_elo_predictions.csv')

# ── 2. Model pickle ──
final_model.save_model('output/models/elo/best_elo')
print('✅ output/models/elo/best_elo.pkl + .json')

# ── 3. Pipeline summary JSON ──
summary = {
    'model': 'Elo',
    'grid_search_configs': len(results_df),
    'fine_sweep_configs': len(fine_df),
    'best_params': best_params,
    'test_metrics': {k: round(v, 6) if isinstance(v, float) else v
                     for k, v in final_metrics.items()},
    'baseline_comparison': {
        'baseline_ensemble_rmse': 1.8071,
        'elo_rmse': round(final_metrics['combined_rmse'], 4),
        'improvement': round(1.8071 - final_metrics['combined_rmse'], 4),
    },
    'team_rankings': {team: round(r, 1) for team, r in final_model.get_rankings()},
    'predictions': preds,
}

with open('output/predictions/elo/elo_pipeline_summary.json', 'w') as f:
    json.dump(summary, f, indent=2)
print('✅ output/predictions/elo/elo_pipeline_summary.json')

print(f'\n🏆 FINAL SUMMARY')
print(f'   Combined RMSE: {final_metrics["combined_rmse"]:.4f}')
print(f'   Win Accuracy:  {final_metrics["win_accuracy"]:.1%}')
print(f'   Best params:   k={best_params["k_factor"]}, ha={best_params["home_advantage"]}, '
      f'mov={best_params["mov_multiplier"]}, rest={best_params["rest_advantage_per_day"]}, '
      f'b2b={best_params["b2b_penalty"]}')
print(f'   vs Baseline:   {"BETTER" if final_metrics["combined_rmse"] < 1.8071 else "NOT YET BEATING"} (1.8071)')